In [6]:
import pandas as pd

pd.options.display.max_colwidth = 500

compute_samples = pd.read_csv('../sourmash/lung_cancer_v4/compute/samples.csv')
print(compute_samples.shape)
compute_samples.head()

/home/ubuntu/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ubuntu/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


(5054, 10)


,id,read1,read2,name,output,trim_low_abundance_kmers,dna,protein,ksizes,scaled
0,A10_B000419_S34,s3://czbiohub-seqbot/fastqs/180516_A00111_0149_AH5CM2DSXX/rawdata/A10_B000419_S34/A10_B000419_S34_R1_001.fastq.gz,s3://czbiohub-seqbot/fastqs/180516_A00111_0149_AH5CM2DSXX/rawdata/A10_B000419_S34/A10_B000419_S34_R2_001.fastq.gz,A10_B000419_S34,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000419_S34.signature,True,True,True,"21,27,33,51",1000
1,A10_B000420_S82,s3://czbiohub-seqbot/fastqs/180516_A00111_0149_AH5CM2DSXX/rawdata/A10_B000420_S82/A10_B000420_S82_R1_001.fastq.gz,s3://czbiohub-seqbot/fastqs/180516_A00111_0149_AH5CM2DSXX/rawdata/A10_B000420_S82/A10_B000420_S82_R2_001.fastq.gz,A10_B000420_S82,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000420_S82.signature,True,True,True,"21,27,33,51",1000
2,A10_B002073_S166,s3://czbiohub-seqbot/fastqs/180516_A00111_0149_AH5CM2DSXX/rawdata/A10_B002073_S166/A10_B002073_S166_R1_001.fastq.gz,s3://czbiohub-seqbot/fastqs/180516_A00111_0149_AH5CM2DSXX/rawdata/A10_B002073_S166/A10_B002073_S166_R2_001.fastq.gz,A10_B002073_S166,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B002073_S166.signature,True,True,True,"21,27,33,51",1000
3,A10_B002078_S202,s3://czbiohub-seqbot/fastqs/180516_A00111_0149_AH5CM2DSXX/rawdata/A10_B002078_S202/A10_B002078_S202_R1_001.fastq.gz,s3://czbiohub-seqbot/fastqs/180516_A00111_0149_AH5CM2DSXX/rawdata/A10_B002078_S202/A10_B002078_S202_R2_001.fastq.gz,A10_B002078_S202,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B002078_S202.signature,True,True,True,"21,27,33,51",1000
4,A10_B002095_S118,s3://czbiohub-seqbot/fastqs/180516_A00111_0149_AH5CM2DSXX/rawdata/A10_B002095_S118/A10_B002095_S118_R1_001.fastq.gz,s3://czbiohub-seqbot/fastqs/180516_A00111_0149_AH5CM2DSXX/rawdata/A10_B002095_S118/A10_B002095_S118_R2_001.fastq.gz,A10_B002095_S118,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B002095_S118.signature,True,True,True,"21,27,33,51",1000


In [7]:
prefix = 's3://olgabot-maca/facs/sourmash_index_all'
txt = 'sourmash_databases.txt'

! aws s3 ls $prefix/ > $txt
! cat $txt

                           PRE tabula-muris-k21-dna/
                           PRE tabula-muris-k21-protein/
                           PRE tabula-muris-k27-dna/
                           PRE tabula-muris-k27-protein/
                           PRE tabula-muris-k33-dna/
                           PRE tabula-muris-k33-protein/
                           PRE tabula-muris-k51-dna/
                           PRE tabula-muris-k51-protein/


In [45]:
# database_name = 'tabula-muris-k51-protein'

# database = f's3://olgabot-maca/facs/sourmash_index_all/{database_name}/{database_name}/'

In [46]:
! aws s3 ls $database

                           PRE .sbt.tabula-muris-k51-protein/
2018-09-13 14:22:45    9225590 tabula-muris-k51-protein.sbt.json


In [47]:
prefix = 's3://olgabot-maca/facs/sourmash_index_all'
txt = 'sourmash_databases.txt'

! aws s3 ls $prefix/ > $txt
! cat $txt

                           PRE tabula-muris-k21-dna/
                           PRE tabula-muris-k21-protein/
                           PRE tabula-muris-k27-dna/
                           PRE tabula-muris-k27-protein/
                           PRE tabula-muris-k33-dna/
                           PRE tabula-muris-k33-protein/
                           PRE tabula-muris-k51-dna/
                           PRE tabula-muris-k51-protein/


In [48]:
databases = pd.read_table(txt, delim_whitespace=True, header=None, names=['is_prefix', 'database_name'])
databases['database_name'] = databases['database_name'].str.strip('/')
databases = databases.drop('is_prefix', axis=1)
databases['ksize'] = databases['database_name'].str.extract('k(\d+)').astype(int)
databases['sequence_to_compare'] = databases['database_name'].map(lambda x: x.split('-')[-1])
databases['database'] = databases['database_name'].map(lambda x: f'{prefix}/{x}/{x}/')
databases = databases.set_index('database_name')
databases

,ksize,sequence_to_compare,database
database_name,,,
tabula-muris-k21-dna,21,dna,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k21-dna/tabula-muris-k21-dna/
tabula-muris-k21-protein,21,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k21-protein/tabula-muris-k21-protein/
tabula-muris-k27-dna,27,dna,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k27-dna/tabula-muris-k27-dna/
tabula-muris-k27-protein,27,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k27-protein/tabula-muris-k27-protein/
tabula-muris-k33-dna,33,dna,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k33-dna/tabula-muris-k33-dna/
tabula-muris-k33-protein,33,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k33-protein/tabula-muris-k33-protein/
tabula-muris-k51-dna,51,dna,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k51-dna/tabula-muris-k51-dna/
tabula-muris-k51-protein,51,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k51-protein/tabula-muris-k51-protein/


In [49]:
protein_databases = databases.query('sequence_to_compare == "protein"')
protein_databases.head()

,ksize,sequence_to_compare,database
database_name,,,
tabula-muris-k21-protein,21,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k21-protein/tabula-muris-k21-protein/
tabula-muris-k27-protein,27,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k27-protein/tabula-muris-k27-protein/
tabula-muris-k33-protein,33,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k33-protein/tabula-muris-k33-protein/
tabula-muris-k51-protein,51,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k51-protein/tabula-muris-k51-protein/


In [56]:
import itertools

ignore_abundances = True, False

data = list(itertools.product(ignore_abundances, compute_samples['output'], 
                              protein_databases.index))

samples = pd.DataFrame(data, columns=['ignore_abundance', 'signature', 'database_name'])
print(samples.shape)
samples.head()

(40432, 3)


,ignore_abundance,signature,database_name
0,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000419_S34.signature,tabula-muris-k21-protein
1,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000419_S34.signature,tabula-muris-k27-protein
2,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000419_S34.signature,tabula-muris-k33-protein
3,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000419_S34.signature,tabula-muris-k51-protein
4,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000420_S82.signature,tabula-muris-k21-protein


In [57]:
import os

samples['sample_id'] = samples['signature'].map(lambda x: os.path.basename(x).split('.')[0])
samples.head()

,ignore_abundance,signature,database_name,sample_id
0,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000419_S34.signature,tabula-muris-k21-protein,A10_B000419_S34
1,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000419_S34.signature,tabula-muris-k27-protein,A10_B000419_S34
2,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000419_S34.signature,tabula-muris-k33-protein,A10_B000419_S34
3,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000419_S34.signature,tabula-muris-k51-protein,A10_B000419_S34
4,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000420_S82.signature,tabula-muris-k21-protein,A10_B000420_S82


In [58]:
samples['id'] = samples.apply(lambda x: 
                              '{sample_id}_ignore-abundance={ignore_abundance}'.format(**x), 
                              axis=1)
samples.head()

,ignore_abundance,signature,database_name,sample_id,id
0,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000419_S34.signature,tabula-muris-k21-protein,A10_B000419_S34,A10_B000419_S34_ignore-abundance=True
1,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000419_S34.signature,tabula-muris-k27-protein,A10_B000419_S34,A10_B000419_S34_ignore-abundance=True
2,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000419_S34.signature,tabula-muris-k33-protein,A10_B000419_S34,A10_B000419_S34_ignore-abundance=True
3,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000419_S34.signature,tabula-muris-k51-protein,A10_B000419_S34,A10_B000419_S34_ignore-abundance=True
4,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000420_S82.signature,tabula-muris-k21-protein,A10_B000420_S82,A10_B000420_S82_ignore-abundance=True


In [59]:
output_prefix = f's3://olgabot-maca/lung_cancer/sourmash_search/{database_name}/'

samples['output'] = output_prefix + samples['id'] + '.csv'

In [61]:
samples_databases = samples.join(protein_databases, on='database_name')
print(samples_databases.shape)
samples_databases.head()

(40432, 9)


,ignore_abundance,signature,database_name,sample_id,id,output,ksize,sequence_to_compare,database
0,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000419_S34.signature,tabula-muris-k21-protein,A10_B000419_S34,A10_B000419_S34_ignore-abundance=True,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k51-protein/A10_B000419_S34_ignore-abundance=True.csv,21,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k21-protein/tabula-muris-k21-protein/
1,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000419_S34.signature,tabula-muris-k27-protein,A10_B000419_S34,A10_B000419_S34_ignore-abundance=True,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k51-protein/A10_B000419_S34_ignore-abundance=True.csv,27,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k27-protein/tabula-muris-k27-protein/
2,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000419_S34.signature,tabula-muris-k33-protein,A10_B000419_S34,A10_B000419_S34_ignore-abundance=True,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k51-protein/A10_B000419_S34_ignore-abundance=True.csv,33,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k33-protein/tabula-muris-k33-protein/
3,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000419_S34.signature,tabula-muris-k51-protein,A10_B000419_S34,A10_B000419_S34_ignore-abundance=True,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k51-protein/A10_B000419_S34_ignore-abundance=True.csv,51,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k51-protein/tabula-muris-k51-protein/
4,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000420_S82.signature,tabula-muris-k21-protein,A10_B000420_S82,A10_B000420_S82_ignore-abundance=True,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k51-protein/A10_B000420_S82_ignore-abundance=True.csv,21,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k21-protein/tabula-muris-k21-protein/


In [31]:
# samples['sequence_to_compare'] = 'protein'
# samples['ksize'] = 51
# samples['database'] = database
# samples['database_name'] = database_name
# samples.head()

,ignore_abundance,signature,sample_id,id,sequence_to_compare,ksize,database,database_name,output
0,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000419_S34.signature,A10_B000419_S34,A10_B000419_S34_ignore-abundance=True,protein,51,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k51-protein/tabula-muris-k51-protein/,tabula-muris-k51-protein,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k51-protein/A10_B000419_S34_ignore-abundance=True.csv
1,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000420_S82.signature,A10_B000420_S82,A10_B000420_S82_ignore-abundance=True,protein,51,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k51-protein/tabula-muris-k51-protein/,tabula-muris-k51-protein,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k51-protein/A10_B000420_S82_ignore-abundance=True.csv
2,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B002073_S166.signature,A10_B002073_S166,A10_B002073_S166_ignore-abundance=True,protein,51,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k51-protein/tabula-muris-k51-protein/,tabula-muris-k51-protein,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k51-protein/A10_B002073_S166_ignore-abundance=True.csv
3,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B002078_S202.signature,A10_B002078_S202,A10_B002078_S202_ignore-abundance=True,protein,51,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k51-protein/tabula-muris-k51-protein/,tabula-muris-k51-protein,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k51-protein/A10_B002078_S202_ignore-abundance=True.csv
4,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B002095_S118.signature,A10_B002095_S118,A10_B002095_S118_ignore-abundance=True,protein,51,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k51-protein/tabula-muris-k51-protein/,tabula-muris-k51-protein,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k51-protein/A10_B002095_S118_ignore-abundance=True.csv


In [75]:
chosen_ids = ['C14_B003528_S62',
 'D1_B003125_S25',
 'E19_B003570_S199',
 'F21_B000420_S213',
 'G10_B003586_S142',
 'G4_B003570_S232',
 'G9_B003511_S57',
 'H7_B003588_S211',
 'I22_B002095_S22',
 'I3_B003573_S63',
 'J11_B003573_S95',
 'J8_B003528_S224',
 'K7_B002073_S103',
 'L16_B003588_S16',
 'L5_B003588_S5',
 'M1_B000420_S61',
 'M23_B002097_S251',
 'N15_B000420_S99',
 'O3_B003573_S207',
 'P14_B000420_S146',
 'P2_B003125_S14']

samples_subset = samples_databases.query('sample_id in @chosen_ids')
print(samples_subset.shape)
samples_subset.head()

(168, 9)


,ignore_abundance,signature,database_name,sample_id,id,output,ksize,sequence_to_compare,database
2784,True,s3://olgabot-maca/lung_cancer/sourmash_v4/C14_B003528_S62.signature,tabula-muris-k21-protein,C14_B003528_S62,C14_B003528_S62_ignore-abundance=True,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k51-protein/C14_B003528_S62_ignore-abundance=True.csv,21,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k21-protein/tabula-muris-k21-protein/
2785,True,s3://olgabot-maca/lung_cancer/sourmash_v4/C14_B003528_S62.signature,tabula-muris-k27-protein,C14_B003528_S62,C14_B003528_S62_ignore-abundance=True,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k51-protein/C14_B003528_S62_ignore-abundance=True.csv,27,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k27-protein/tabula-muris-k27-protein/
2786,True,s3://olgabot-maca/lung_cancer/sourmash_v4/C14_B003528_S62.signature,tabula-muris-k33-protein,C14_B003528_S62,C14_B003528_S62_ignore-abundance=True,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k51-protein/C14_B003528_S62_ignore-abundance=True.csv,33,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k33-protein/tabula-muris-k33-protein/
2787,True,s3://olgabot-maca/lung_cancer/sourmash_v4/C14_B003528_S62.signature,tabula-muris-k51-protein,C14_B003528_S62,C14_B003528_S62_ignore-abundance=True,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k51-protein/C14_B003528_S62_ignore-abundance=True.csv,51,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k51-protein/tabula-muris-k51-protein/
4368,True,s3://olgabot-maca/lung_cancer/sourmash_v4/D1_B003125_S25.signature,tabula-muris-k21-protein,D1_B003125_S25,D1_B003125_S25_ignore-abundance=True,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k51-protein/D1_B003125_S25_ignore-abundance=True.csv,21,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k21-protein/tabula-muris-k21-protein/


In [76]:
samples_no_sample_id = samples_subset.drop(columns=['sample_id'])
samples_no_sample_id = samples_no_sample_id.set_index('id')
print(samples_no_sample_id.shape)
samples_no_sample_id.head()

(168, 7)


,ignore_abundance,signature,database_name,output,ksize,sequence_to_compare,database
id,,,,,,,
C14_B003528_S62_ignore-abundance=True,True,s3://olgabot-maca/lung_cancer/sourmash_v4/C14_B003528_S62.signature,tabula-muris-k21-protein,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k51-protein/C14_B003528_S62_ignore-abundance=True.csv,21,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k21-protein/tabula-muris-k21-protein/
C14_B003528_S62_ignore-abundance=True,True,s3://olgabot-maca/lung_cancer/sourmash_v4/C14_B003528_S62.signature,tabula-muris-k27-protein,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k51-protein/C14_B003528_S62_ignore-abundance=True.csv,27,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k27-protein/tabula-muris-k27-protein/
C14_B003528_S62_ignore-abundance=True,True,s3://olgabot-maca/lung_cancer/sourmash_v4/C14_B003528_S62.signature,tabula-muris-k33-protein,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k51-protein/C14_B003528_S62_ignore-abundance=True.csv,33,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k33-protein/tabula-muris-k33-protein/
C14_B003528_S62_ignore-abundance=True,True,s3://olgabot-maca/lung_cancer/sourmash_v4/C14_B003528_S62.signature,tabula-muris-k51-protein,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k51-protein/C14_B003528_S62_ignore-abundance=True.csv,51,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k51-protein/tabula-muris-k51-protein/
D1_B003125_S25_ignore-abundance=True,True,s3://olgabot-maca/lung_cancer/sourmash_v4/D1_B003125_S25.signature,tabula-muris-k21-protein,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k51-protein/D1_B003125_S25_ignore-abundance=True.csv,21,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k21-protein/tabula-muris-k21-protein/


In [77]:
# folder = f'../sourmash/lung_cancer_v4/search_{database_name}'
folder = f'../sourmash/lung_cancer_v4/search_protein_databases'
! mkdir $folder

mkdir: cannot create directory ‘../sourmash/lung_cancer_v4/search_protein_databases’: File exists


In [78]:
import json

config = 	{
		"program": "../../../reflow/sourmash_search.rf",
		"runs_file": "samples.csv"
	}

samples_no_sample_id.to_csv(f'{folder}/samples.csv', index=True)


with open(f'{folder}/config.json', 'w') as f:
    json.dump(config, f)

In [79]:
pwd

'/home/ubuntu/kmer-hashing/notebooks'

In [74]:
ls -lha $folder

total 60K
drwxrwxr-x 2 ubuntu ubuntu 4.0K Sep 21 02:28 ./
drwxrwxr-x 6 ubuntu ubuntu 4.0K Sep 21 02:28 ../
-rw-rw-r-- 1 ubuntu ubuntu   77 Sep 21 02:28 config.json
-rw-rw-r-- 1 ubuntu ubuntu  48K Sep 21 02:28 samples.csv
